In [3]:
from flask import Flask,request,jsonify
import os
from langchain_community.document_loaders import PyPDFLoader,TextLoader,Docx2txtLoader
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import db2
from datetime import datetime

app = Flask(__name__)

conn,cursor = db2.connection(app)

UPLOAD_FOLDER = "Uploads"
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route('/register', methods=['POST'])
def register():
    name = request.form['name']
    email = request.form['email']
    password = request.form['password']
    confirm = request.form['confirm']
    error = None

    if not (8 <= len(name) <= 20):
        error = 'Username length must be between 8 and 20 characters'
    
    if not (8 <= len(password) <= 20):
        error = 'Password length must be between 8 and 20 characters'

    if not re.match(r'^[\w\.-]+@[\w\.-]+$', email):
        error = 'Invalid email format'
    
    if password != confirm:
        error = 'password and confirm password does not match'
    else:
        cursor.execute('SELECT * FROM registrationtable WHERE email=%s', (email,))
        user = cursor.fetchone()
        if user:
            error = 'Sorry, email already exist!'

    cursor.execute("INSERT INTO registrationtable (Name,Email,Password,Confirm) VALUES (%s, %s, %s, %s)",
                       (name,email,password,confirm))
    conn.commit()

    if error is None:
        return jsonify({"output": "you are successfully registered"})
    else:
        return jsonify({"error":error})
    

@app.route('/login', methods=['POST'])
def login():
    email = request.form['email']
    password = request.form['password']
    error = None
    cursor.execute('SELECT * FROM registrationtable WHERE email=%s', (email,))
    user = cursor.fetchone()
    if user is None:
        error = 'Incorrect username.'
    elif user[2]!= password:
        error = 'Incorrect password.'
        
    if error is None:
        return jsonify({"output":"Login Successful"})
    else:
        return jsonify({"error":error})

@app.route("/uploads",methods = ["POST"])
def summarization():
    name = request.form['name']
    file = request.files["file"]

    if file == "":
        return jsonify({"error": "Please upload file"})
    if file:
        file_name = file.filename
        file_extension = file.path.splittext(filename)[1].lower()
        #file_path = os.path.join(app.config['UPLOAD_FOLDER'],file_name)
        #os.save(file_path)
    if file_extension == ".pdf":
        loader = PyPDFLoader()
    if file_extension == ".docx":
        loader = Docx2txtLoader()
    if file_extension == ".txt":
        loader = TextLoader()
    data = loader.load()
    template = """ You have to summarize the data i am giving to you. 
    It is a tender document. So,do not miss any important points.
    Observe entire document carefully and summarize it.
    
    {data}
    """
    prompt = PromptTemplate(template = template,
                            input = ["data"])
    final_prompt = prompt.format(data = data)
    model = ChatOpenAI()
    summary = model.invoke(final_prompt)
    date = current_datetime.date()
    time = current_datetime.time()
    cursor.execute("INSERT INTO summarizationtable (Name,file_name,date,time) VALUES (%s, %s, %s, %s)",
                       (name,file_name,date,time))
    conn.commit()
    return jsonify({"Summary" : summary})
    
    
    
if __name__ =="__main__":  
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [09/May/2024 16:46:03] "POST /uploads HTTP/1.1" 400 -


In [ ]:
from langchain_openai import ChatOpenAI
from